### Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
# Most code adapted from Gabriel Weindel at https://github.com/GWeindel/hmp
# https://github.com/GWeindel/hmp/blob/main/tutorials/1-Data_loading.ipynb
import hmp
from pathlib import Path
import os
DATA_PATH = Path(os.getenv("DATA_PATH"))

### Set up paths and participants

In [ ]:
# Path where pre-processed EEG data can be found
eeg_data_path = DATA_PATH / "sat1/preprocessed_500hz/"

# Path where output will be saved
output_filename = "data_250hz.nc"
output_path = DATA_PATH / "sat1" / output_filename

# Change array indexing when you want to create a dataset consisting of a subset of the participants
subj_ids = [
    (
        subj_id.name.split("_")[2]
        if "500hz" in str(eeg_data_path)
        else subj_id.name.split("_")[1]
    )
    for subj_id in eeg_data_path.glob("*.fif")
]
subj_ids.sort()
subj_files = [
    (
        str(eeg_data_path / f"processed_500Hz_{subj_id}_epo.fif")
        if "500hz" in str(eeg_data_path)
        else str(eeg_data_path / f"processed_{subj_id}_epo.fif")
    )
    for subj_id in subj_ids
]
print(subj_ids)

### Load data into epoched xarray and save

In [ ]:
data = hmp.utils.read_mne_data(
    subj_files,
    epoched=True,
    sfreq=250,
    scale=1000,
    lower_limit_RT=0.2,
    upper_limit_RT=1.5, # 1.5s since any trial longer than this is 'no answer' according to Boehm 2014
    verbose=False,
    subj_idx=subj_ids,
    rt_col='RT',
    offset_after_resp=0.25,
    offset_before_stim=0.25,
)
tmp_offset = data.offset
data = data.assign_attrs({'offset': 0, 'extra_offset': tmp_offset})
data.to_netcdf(output_path)